# MARSIS xDR-RAW Read To Export (R2E)
@author: Giacomo Nodjoumi g.nodjoumi@jacobs-unversity.de

## Create and activate the environment using the yml

Using the terminal, craate conda env using provided MARSISv2.yml:
* `conda env create -f MARSISv2.yml`
* `conda activate MARSISv2`

## Arguments requested to user:
* Output Directory: path where all files are saved
* Data directory: path where are all files to be processed
* Driver for gis output: insert choice between GPKG, gpkg, SHP, shp
* Data record type: insert choice between EDR, edr, RDR, rdr, RAW, raw
* Flag to save images of both frequencies: insert choice between Y,y,N,n
* Flag to save numpy dump of both frequencies: insert choice between Y,y,N,n
* Flag to save seg-y of both frequencies: insert choice between Y,y,N,n

## Outputs:
### GIS OUTPUTS
It creates three different geopackages:
    - FULL with all orbits
    - North Pole with orbits from 65°->90° Latitude
    - South Pole with orbits from -65°->-90° Latitude
### Image outputs
As default it creates thre types of images for each frequency:
* Original image
* Normalized image
* Scaled image using sklearn MinMaxScaler

*Further image processing is in development*
### SEG-Y outputs
It export a seg-y file for each frequency of each image

# Import package and sub-modules

In [ ]:
import numpy as np
import os
import pathlib
import geopandas as gpd
import pandas as pd
from tqdm.auto import tqdm
from statistics import mean
from shapely.geometry import LineString
from pyproj.crs import CRS
from pyproj import Transformer
from ipywidgets import IntProgress
import re
import itertools
from utils.FileUtils import DAT2FILE
from utils.GenUtils import get_paths, make_folder, question
from utils.ReprojUtils import coordTransformer
from utils.DFUtils import DF_drop, geoDF2file, gdf_split, xDR_DF
from utils.RawUtils import raw_reader
from utils.SegyUtils import assemply_segy, save_segy
from utils.DBUtils import databaseUpdate

# Define CRS for global, North and South poles of MARS

In [ ]:
N_pole_crs = CRS.from_proj4('+proj=stere +lat_0=90 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=3396190 +b=3376200 +units=m +no_defs ')
S_pole_crs = CRS.from_proj4('+proj=stere +lat_0=-90 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=3396190 +b=3376200 +units=m +no_defs ')
marsPLNTC = CRS.from_proj4('+proj=longlat +a=3396190 +b=3376200 +no_defs')
marsPLANCE = CRS.from_user_input('GEOGCS["Mars 2000 planetocentric",DATUM["D_Mars_2000",SPHEROID["Mars_2000_IAU_IAG",3396190.0,169.89444722361179]],PRIMEM["AIRY-0",180],UNIT["Degree",0.017453292519943295]]')
marsPLANCE180 = CRS.from_user_input('GEOGCS["Mars 2000 planetocentric",DATUM["D_Mars_2000",SPHEROID["Mars_2000_IAU_IAG",3396190.0,169.89444722361179]],PRIMEM["AIRY-0",180],UNIT["Degree",0.017453292519943295]]')
marsEQUI180 = CRS.from_user_input('PROJCS["Mars_Equidistant_Cylindrical",GEOGCS["Mars 2000",DATUM["D_Mars_2000",SPHEROID["Mars_2000_IAU_IAG",3396190.0,169.89444722361179]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Equidistant_Cylindrical"],PARAMETER["False_Easting",0],PARAMETER["False_Northing",0],PARAMETER["Central_Meridian",180],PARAMETER["Standard_Parallel_1",0],UNIT["Meter",1]]')
marsUTM = CRS.from_proj4('+proj=tmerc +lat_0=0 +lon_0=0 +k=0.9996 +x_0=0 +y_0=0 +a=3396190 +b=3376200 +units=m +no_defs')
marsMRCA = CRS.from_user_input('PROJCS["Mars_Mercator_AUTO",GEOGCS["Mars 2000",DATUM["D_Mars_2000",SPHEROID["Mars_2000_IAU_IAG",3396190.0,169.89444722361179]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Mercator"],PARAMETER["False_Easting",0],PARAMETER["False_Northing",0],PARAMETER["Central_Meridian",0],PARAMETER["Standard_Parallel_1",0],UNIT["Meter",1]]')
mars2000=CRS.from_wkt('GEOGCS["Mars 2000",DATUM["D_Mars_2000",SPHEROID["Mars_2000_IAU_IAG",3396190.0,169.89444722361179]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]')

# Core function
This is the core function that read the binary data file and convert it into:
- a complete dataframe
- a complete geodataframe 

In [ ]:
def RAW2GeoDF(xDR_df, xDR_gdf, xDrFile, ParamDF, def_crs):
    fname = pathlib.Path(xDrFile).name.split('.')[0]
    #F_NAME = re.split('_|\.', fname)[0:6]
    full_parameters = [] #All values for each parameter
    short_parameters = [] #Mean value for each parameter
    

    for i in range(len(ParamDF['NAME'])):
       # print(ParamDF['NAME'][i])
        offBytes=ParamDF['START_BYTES'][i]
        precision = ParamDF['DATA_TYPE'][i]
        Items = ParamDF['ITEMS'][i]
        ItemBytes=ParamDF['ITEM_BYTES'][i]
        file = open(xDrFile, 'rb')
        FileSize = pathlib.Path(xDrFile).stat().st_size
        RecordBytes=ParamDF['RECORD_BYTES'][0]
        FileRecord = int(FileSize/RecordBytes)
        f = file.read()
        if Items > 1 and 'S' not in precision:
            values=[]
            mean_values = []
            for l in range(Items):
                Item = l*ItemBytes
                val = np.array(raw_reader(f,offBytes,i,FileRecord,Item,RecordBytes, precision))
                values.append(val)
                mean_values.append(mean(val))
        else:
            Item=0
            values = np.array(raw_reader(f,offBytes,i,FileRecord,Item,RecordBytes, precision))
            if 'S' in precision:
                mean_values= values[0]
            else:
                mean_values=mean(values)
        full_parameters.append(values)
        short_parameters.append(mean_values)
        file_name=fname.split('.')[0]
        if ParamDF['NAME'][i] in ['SUB_SC_LONGITUDE','SUB_SC_LATITUDE']:
            min_val = min(values)
            max_val = max(values)
            short_parameters.append(min_val)
            short_parameters.append(max_val)
           # print(min_val, ' ', max_val)

    if DRTYPE in ['RDR', 'rdr','EDR','edr']:
        a, b,c,d = [11,17,29,30]
        F= [full_parameters[a], full_parameters[b]]
        coord = list(zip(full_parameters[c], full_parameters[d]))
        lat_mean = mean(full_parameters[d])
        dt_value = 7.14285714285714e-03
        scl = -10
        
        if lat_mean <=-65:
            segy_crs = S_pole_crs
        elif lat_mean >=65:
            segy_crs = N_pole_crs
        else:
            segy_crs = marsEQUI180
            
    elif DRTYPE in ['RAW','raw']:
        a, b,c,d = [9,11,21,22]
        F= [full_parameters[a], full_parameters[b]]
        coord = list(zip(full_parameters[c], full_parameters[d]))
        lat_mean = mean(full_parameters[d])
        scl = -1
        dt_value = 7.14285714285714e-03
        
        if lat_mean <=-65:
            segy_crs = S_pole_crs
        elif lat_mean >=65:
            segy_crs = N_pole_crs
        else:
            segy_crs = marsEQUI180

    segy_coord = coordTransformer(coord, def_crs, segy_crs)            
    DAT2FILE(IMG_DIR, DUMP_DIR, SEGY_DIR, file_name, F, SAVEDUMP, SAVEIMG, SAVESEGY,
             segy_coord, dt=dt_value ,pp='y',scaler=scl)
    
    #dst_crs = marsEQUI180
    dst_crs = DST_CRS
    # Create the linear geometry
    trans_coord = coordTransformer(coord, def_crs, dst_crs)
    track =  LineString(trans_coord)
    # create the geoSeries
    gser = gpd.GeoSeries(track)
    
    # Create a dataframe containing all values for each parameter
    proj4= dst_crs.to_wkt()
    meta = [fname, proj4]
    temp_full = meta+full_parameters
    series_full = pd.Series(temp_full, index=xDR_df.columns)
    xDR_df = xDR_df.append(series_full,ignore_index=True)
    # Create a temporary dataframe containing all mean values for each parameter
    temp_short = meta+short_parameters
    
    if DRTYPE in ['RDR','rdr']:
        aa, bb, cc, dd = [32,33,35,36]
    else:
        aa, bb, cc, dd = [22,23,25,26]
        
    short_cols = xDR_df.columns.tolist()
    short_cols.insert(aa,'c1min')
    short_cols.insert(bb,'c1max')
    short_cols.insert(cc,'c2min')
    short_cols.insert(dd,'c2max')
    
    series_short = pd.Series(temp_short, index=short_cols)
    df = pd.DataFrame(columns=short_cols).append(series_short, ignore_index=True)
    # Create the geodataframe containing all mean values and geometry for each parameter
    xDR_geodf = gpd.GeoDataFrame(df, crs = dst_crs, geometry=gser)
    return(xDR_geodf, xDR_df, coord, trans_coord, segy_coord)

Functions related to parallelization and chunk creation

In [ ]:
def parallel_df(files, JOBS,FM_df, FM_gdf, GeomDF, def_crs):
    from joblib import Parallel, delayed
    results = Parallel (n_jobs=JOBS)(delayed(RAW2GeoDF)(FM_df, FM_gdf, files[i], GeomDF, def_crs)
                            for i in range(len(files)))
    return (results)

def chunk_creator(item_list, chunksize):
    it = iter(item_list)
    while True:
        chunk = tuple(itertools.islice(it, chunksize))
        if not chunk:
            break
        yield chunk

# Main function
The main function do:
- create a list of all files to be processed
- evaluate available RAM and CPU and scale CPU cores
- create chunks for smoother parallel processing
- load sub-modules for xDR/RAW
- initialize all dataframes
- drop from geodataframe all the parameter with incompatible 3-dimensional parameters (e.g. MONOPOLE_UNIT_VECTOR)
- save complete geopackage or shapefile
- filter data of poles and save corresponding geopackage or shapefile

In [ ]:
def main():        
    # List all files
    all_filenames = get_paths(DATA_PATH, 'dat')
    
    # Check available resources
    import psutil
    avram=psutil.virtual_memory().total >> 3
    if avram > 31 and len(all_filenames) <5000:
        JOBS=psutil.cpu_count(logical=True)
    elif avram > 31 and len(all_filenames)>5000:
        JOBS=psutil.cpu_count(logical=True)
    elif avram <=31 and len(all_filenames)<5000:
        JOBS=psutil.cpu_count(logical=True)
    elif avram <= 31 and len(all_filenames) > 5000:
        JOBS=psutil.cpu_count(logical=False)
    
    # Create chunks for parallel processing
    filerange = len(all_filenames)
    chunksize = round(filerange/JOBS)
    if chunksize <1:
        chunksize=1
        JOBS = filerange
    chunks = []
    for c in chunk_creator(all_filenames, JOBS):
        chunks.append(c)
               
    # Load sub-functions
    if DRTYPE in ['RDR', 'rdr','EDR','edr']:
        from utils.DFUtils import xDR_params
        def_crs = marsPLNTC
        ParamDF=xDR_params()
    
    elif DRTYPE in ['RAW','raw']:
        from utils.DFUtils import RAW_params
        def_crs = marsPLNTC
        ParamDF = RAW_params()
        
    # Initialize dataframes
    xDR_df = xDR_DF(ParamDF)
    xDR_gdf = gpd.GeoDataFrame(xDR_df)
    results = []
    
    # Parallel processing
    with tqdm(total=len(all_filenames),
             desc = 'Generating files',
             unit='File') as pbar:
        
        for i in range(len(chunks)):
            files = chunks[i]    
            # print(files)
            chunk_results = parallel_df(files, JOBS,xDR_df, xDR_gdf, ParamDF, def_crs)
            
            for r in chunk_results:
                results.append(r)
                xDR_df = pd.concat([xDR_df, r[1]])
                xDR_gdf = pd.concat([xDR_gdf, r[0]])
            pbar.update(JOBS)

    # Sort dataframe elements by orbit number and drop incompatible Parameters from dataframe
    xDR_df = xDR_df.sort_values(by='name', ascending=True, ignore_index=True)    
    xDR_gdf = xDR_gdf.sort_values(by='name', ascending=True, ignore_index=True)    
    xDR_gdf = DF_drop(xDR_gdf, DRTYPE)
    if DRVR is None:
        print('All Done')
    else:        
        print('\nSaving Full geopackate')    
        # geoDF2file(xDR_gdf, 'Complete', marsEQUI180, SAVE_DIR, DRVR)
        geoDF2file(xDR_gdf, 'Complete', [DST_CRS], SAVE_DIR, DRVR)
        print('\nSaving N-Pole geopackage')
        xDR_gdf_NPole = gdf_split(xDR_gdf, 65)
        geoDF2file(xDR_gdf_NPole, 'NPole', [DST_CRS, N_pole_crs], SAVE_DIR, DRVR)
        print('\nSaving S-Pole geopackage')
        xDR_gdf_SPole = gdf_split(xDR_gdf, -65)
        geoDF2file(xDR_gdf_SPole, 'SPole', [DST_CRS, S_pole_crs], SAVE_DIR, DRVR)
        
    if DBUP in ['Y','y']:
        error = databaseUpdate(xDR_gdf)
        if error == False:
            print('Database updated')
        else:
            print(error)
    return(xDR_gdf)

# User interactive inputs
Ask user for various input:
- General output directory
- Data directory
- Type of file to be saved
- Type of file ingested
- Save geopackage/shapefile
- Save images of both frequencies
- Save dumps of both frequencies
- Save Seg-y file for both frequency

In [ ]:
if __name__ == "__main__":

    WORK_PATH = input(str('Path of the output folder '))
                
    DATA_PATH = input(str('Path to data files folder:'))
        
    DRTYPE = question('Enter data type: EDR/RDR/RAW',['EDR','edr','RDR','rdr','RAW','raw'])
    
    qst = question('Save geopackagse/shapefiles',  ['Y','y','N','n'])
    if qst in ['Y','y']:
        DRVR= question('Enter output file type: GPKG/gpkg or SHP/shp', ['GPKG','gpkg','SHP','shp'])
    else:
        DRVR = None
    SAVEIMG = question('Save images?', ['Y','y','N','n'])
    
    SAVEDUMP = question('Dump arrays?', ['Y','y','N','n'])

    SAVESEGY = question('Save Seg-y', ['Y','y','N','n'])
    
    
    if SAVEIMG in ['Y', 'y']:
        IMG_DIR = make_folder(WORK_PATH,'Images')
    else:
        IMG_DIR=None
    if SAVEDUMP in ['Y', 'y']:
        DUMP_DIR = make_folder(WORK_PATH, 'Dumps')
    else:
        DUMP_DIR=None
    if SAVESEGY in ['Y', 'y']:
        SEGY_DIR = make_folder(WORK_PATH, 'Seg-y')
    else:
        SEGY_DIR = None
        
    if DRVR in ['GPKG','gpkg']:
        SAVE_DIR = make_folder(WORK_PATH,'Geopackages')
    elif DRVR in ['SHP', 'shp']:
        SAVE_DIR = make_folder(WORK_PATH,'Shapefiles')
    else:
        DRVR = None
    DST_CRS = input('Destination CRS in wkt/proj4 format - Leave empty for Mars Ecquirectangular') or marsEQUI180
    
    #DBUP = question('Save results into postgres database?',['Y','y','N','n'])

    
    main()